In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/",one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
x_raw,y_raw = mnist.train.next_batch(40)
# Constants and variables
n_classes = 10
img_width = 28
img_height = 28

# For ipython
tf.reset_default_graph()


In [3]:
def getModel():
    
     # Network
        x = tf.placeholder("float", [None, img_width*img_height])
        y = tf.placeholder("float", [None, n_classes])

        xi = tf.reshape(x,[-1,28,28,1])
        
       
        with tf.variable_scope("conv1") as scope:
                W = tf.get_variable("W",shape=[3,3,1,32],initializer=tf.contrib.layers.xavier_initializer())
                b = tf.get_variable("b",initializer=tf.zeros([32]))
                l1 = tf.nn.conv2d(xi,W,strides=[1,1,1,1],padding="VALID")
                l1_b = tf.nn.bias_add(l1,b)
                l1_act = tf.nn.relu(l1_b)

        with tf.variable_scope("conv2") as scope:
                W = tf.get_variable("W",shape=[3,3,32,64],initializer=tf.contrib.layers.xavier_initializer())
                b = tf.get_variable("b",initializer=tf.zeros([64]))
                l2 = tf.nn.conv2d(l1_act,W,strides=[1,1,1,1],padding="VALID")
                l2_b = tf.nn.bias_add(l2,b)
                l2_act = tf.nn.relu(l2_b)

        # Maxpooling
        l3_max = tf.nn.max_pool(l2_act,[1,2,2,1],strides=[1,2,2,1],padding="VALID")
        
        with tf.variable_scope("dense1") as scope:
                W = tf.get_variable("W",shape=[12*12*64,128],initializer=tf.contrib.layers.xavier_initializer())
                b = tf.get_variable("b",initializer=tf.zeros([128]))
                l4 = tf.reshape(l3_max,[-1,12*12*64])
                l4 = tf.matmul(l4,W)
                l4_b = tf.nn.bias_add(l4,b)
                l4_act = tf.nn.relu(l4_b)


        with tf.variable_scope("dense2") as scope:
                W = tf.get_variable("W",shape=[128,n_classes],initializer=tf.contrib.layers.xavier_initializer())
                b = tf.get_variable("b",initializer=tf.zeros([n_classes]))
                l5 = tf.matmul(l4,W)+b
                
                
        eval_pred = tf.nn.softmax(l5)
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=l5,labels=y))
        train_step = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

        return (x,y),train_step,cost,eval_pred

# Getting variables